# Agentic AI with Websearch like tools

### Agent with Web Search capability

In [ ]:
import os
from agents import Agent, Runner, function_tool, WebSearchTool
from agents.model_settings import ModelSettings
import datetime
from typing import Any, Dict, List
from IPython.display import display, HTML, clear_output


from dotenv import load_dotenv

load_dotenv()


os.environ["OPENAI_BASE_URL"] = os.getenv("LMSTUDIO_API_BASE_URL", "http://localhost:1234/v1")
os.environ["OPENAI_API_KEY"] = os.getenv("LMSTUDIO_API_KEY", "lm-studio")

openai_model = "openai/gpt-oss-20b"

In [ ]:
WebResearcherAgentInstruction = """
You are a Web Researcher Agent.

You have TWO tools:
1) web_search(query)  [must be called ONCE]
2) html_formatter(claim, verdict, conclusion, evidence[], sources[]) [must be called ONCE AFTER web_search]

PROCESS (MANDATORY):
Step A) Call web_search exactly once to gather results.
Step B) Decide:
  - verdict: VERIFIED / FALSE / UNVERIFIED
  - conclusion: 1 short paragraph
  - evidence: 2–3 bullet strings based on the results
  - sources: 2–5 items: {title, url}
Step C) Call html_formatter exactly once with those fields.
Step D) Return ONLY the string returned by html_formatter. No extra text.

VERDICT RULES:
- VERIFIED: multiple credible sources clearly confirm.
- FALSE: credible sources clearly refute.
- UNVERIFIED: unclear/insufficient/conflicting evidence.

Never call any other tools. Never call web_search twice.
"""

In [ ]:
search_agent = Agent(
    name="Web Researcher Agent",
    instructions=WebResearcherAgentInstruction,
    tools=[WebSearchTool(search_context_size=5)],
    model=openai_model,
    #model_settings=ModelSettings(tool_choice="required")
)

### Tools

#### A perfect News validator program

In [ ]:
import httpx
from ddgs import DDGS
from agents import Agent, Runner, function_tool
import datetime
from typing import List
from pydantic import BaseModel
from agents import function_tool

@function_tool
def web_search(query: str) -> dict:
    """Search the web once and return top results."""
    results = []

    with DDGS() as ddgs:
        for r in ddgs.text(query, max_results=3):
            results.append({
                "title": r.get("title"),
                "url": r.get("href"),
                "snippet": (r.get("body") or "")[:300],
            })

    return {
        "instruction": "Use these results to answer. Do NOT search again.",
        "results": results,
    }

@function_tool
def web_fetch(url: str) -> str:
    """Fetch a URL and return the raw HTML (trimmed)."""
    with httpx.Client(follow_redirects=True, timeout=15) as client:
        html = client.get(url).text
    return html[:15000]  # keep it bounded


class Source(BaseModel):
    title: str
    url: str

@function_tool
def html_formatter(
    claim: str,
    verdict: str,
    conclusion: str,
    evidence: List[str],
    sources: List[Source],   # ✅ no Dict here
) -> str:
    ts = datetime.datetime.utcnow().strftime("%Y-%m-%d %H:%M UTC")
    """
    Format the final report as a complete HTML document.
    verdict must be one of: VERIFIED, FALSE, UNVERIFIED
    sources: list of {title, url}
    """

    verdict = (verdict or "UNVERIFIED").strip().upper()
    styles = {
        "VERIFIED": ("#dcfce7", "#166534"),
        "FALSE": ("#fee2e2", "#991b1b"),
        "UNVERIFIED": ("#fef3c7", "#92400e"),
    }
    bg, fg = styles.get(verdict, styles["UNVERIFIED"])

    ev_items = "\n".join(
        f"<li style='margin:6px 0; line-height:1.5; color:#111827;'>{e}</li>"
        for e in (evidence or [])[:3]
        if e and e.strip()
    ) or "<li style='margin:6px 0; line-height:1.5; color:#6b7280;'>No clear evidence extracted.</li>"

    src_items = "\n".join(
        f"""
        <li style="margin:10px 0;">
          <a href="{s.get('url','')}" style="color:#2563eb;text-decoration:none;font-weight:600;">
            {s.get('title','(source)')}
          </a>
          <div style="font-size:12px;color:#6b7280;word-break:break-all;">{s.get('url','')}</div>
        </li>
        """
        for s in (sources or [])[:5]
        if s and s.get("url")
    ) or "<li style='margin:10px 0; color:#6b7280;'>No sources available.</li>"

    html = f"""<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <title>Claim Verification Report</title>
</head>
<body style="margin:0;padding:0;background:#f6f8fb;font-family:Arial,Helvetica,sans-serif;color:#111827;">
  <div style="max-width:820px;margin:0 auto;padding:24px;">
    <div style="background:#ffffff;border:1px solid #e5e7eb;border-radius:16px;padding:20px;">
      <div style="font-size:12px;color:#6b7280;text-transform:uppercase;letter-spacing:.06em;">Web Research Result</div>
      <div style="margin-top:6px;font-size:22px;font-weight:800;">Claim Verification Report</div>
      <div style="margin-top:8px;font-size:12px;color:#6b7280;">Timestamp (UTC): <span style="color:#374151;font-weight:700;">{ts}</span></div>
    </div>

    <div style="margin-top:14px;background:#ffffff;border:1px solid #e5e7eb;border-radius:16px;padding:20px;">
      <div style="font-size:14px;font-weight:800;margin-bottom:10px;">Claim</div>
      <div style="font-size:14px;line-height:1.6;background:#f9fafb;border:1px solid #eef2f7;border-radius:12px;padding:14px;">
        {claim}
      </div>
    </div>

    <div style="margin-top:14px;background:#ffffff;border:1px solid #e5e7eb;border-radius:16px;padding:20px;">
      <div style="display:flex;align-items:center;gap:10px;flex-wrap:wrap;">
        <div style="font-size:14px;font-weight:800;">Verdict</div>
        <div style="font-size:12px;font-weight:900;padding:6px 12px;border-radius:999px;background:{bg};color:{fg};">
          {verdict}
        </div>
      </div>
      <div style="margin-top:10px;font-size:14px;line-height:1.7;color:#111827;">
        {conclusion}
      </div>
    </div>

    <div style="margin-top:14px;background:#ffffff;border:1px solid #e5e7eb;border-radius:16px;padding:20px;">
      <div style="font-size:14px;font-weight:800;margin-bottom:10px;">Key Evidence</div>
      <ul style="margin:0;padding-left:18px;">{ev_items}</ul>
    </div>

    <div style="margin-top:14px;background:#ffffff;border:1px solid #e5e7eb;border-radius:16px;padding:20px;">
      <div style="font-size:14px;font-weight:800;margin-bottom:10px;">Sources</div>
      <ul style="margin:0;padding-left:18px;">{src_items}</ul>
      <div style="margin-top:12px;font-size:12px;color:#9ca3af;">
        Generated by Web Researcher Agent
      </div>
    </div>
  </div>
</body>
</html>"""
    return html



In [ ]:
from agents import Agent, ModelSettings

search_agent = Agent(
    name="Web Researcher Agent",
    instructions=WebResearcherAgentInstruction,
    tools=[web_search, html_formatter],
    model=openai_model,  # your LM Studio OpenAI-compatible model handle
    model_settings=ModelSettings(
        temperature=0.1,
        max_tool_calls=10,  # web_search then html_formatter
    ),
)

In [ ]:
message = """
Investigate whether the following international development has been officially reported by reputable global media.
Search 2 times and summarize the confirmed facts and format the final output as a complete HTML document.

Claim:
World war 3 is on now.
"""
result = Runner.run_streamed(search_agent, input=message, max_turns=10)
async for event in result.stream_events():
    if event.type == "raw_response_event" and event.data.type == "response.output_text.delta":
            print(event.data.delta, end="", flush=True)

result_html = result.final_output


In [ ]:
from IPython.display import display, HTML
display(HTML(result_html))